In [4]:
 import numpy as np
import pandas as pd


import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import spacy
import re,string
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import WordNetLemmatizer

from sklearn.naive_bayes import MultinomialNB

from textblob import TextBlob
from textblob import Word
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

import os

import warnings
warnings.filterwarnings('ignore')

In [5]:
#importing the training data
imdb_data=pd.read_csv('IMDB Dataset.csv')
print(imdb_data.shape)
imdb_data.head(10)

(50000, 2)


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
5,"Probably my all-time favorite movie, a story o...",positive
6,I sure would like to see a resurrection of a u...,positive
7,"This show was an amazing, fresh & innovative i...",negative
8,Encouraged by the positive comments about this...,negative
9,If you like original gut wrenching laughter yo...,positive


In [6]:
#sentiment count
imdb_data['sentiment'].value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

In [7]:
#Summary of the dataset
imdb_data.describe()

,review,sentiment
count,50000,50000
unique,49582,2
top,Loved today's show!!! It was a variety and not...,positive
freq,5,25000


In [8]:
#split the dataset
#train dataset
train_reviews=imdb_data.review[:40000]
train_sentiments=imdb_data.sentiment[:40000]
#test dataset
test_reviews=imdb_data.review[40000:]
test_sentiments=imdb_data.sentiment[40000:]
print(train_reviews.shape,train_sentiments.shape)
print(test_reviews.shape,test_sentiments.shape)

(40000,) (40000,)
(10000,) (10000,)


In [9]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [10]:
#Tokenization of text
tokenizer=ToktokTokenizer()
#Setting English stopwords
stopword_list=nltk.corpus.stopwords.words('english')

In [11]:
#Removing the html strips
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

#Removing the square brackets
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

#Removing the noisy text
def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    return text
#Apply function on review column
imdb_data['review']=imdb_data['review'].apply(denoise_text)

In [12]:
#Define function for removing special characters
def remove_special_characters(text, remove_digits=True):
    pattern=r'[^a-zA-z0-9\s]'
    text=re.sub(pattern,'',text)
    return text
#Apply function on review column
imdb_data['review']=imdb_data['review'].apply(remove_special_characters)

In [13]:
#Stemming the text
def simple_stemmer(text):
    ps=nltk.porter.PorterStemmer()
    text= ' '.join([ps.stem(word) for word in text.split()])
    return text
#Apply function on review column
imdb_data['review']=imdb_data['review'].apply(simple_stemmer)

In [14]:
#set stopwords to english
stop=set(stopwords.words('english'))
print(stop)

#removing the stopwords
def remove_stopwords(text, is_lower_case=False):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text
#Apply function on review column
imdb_data['review']=imdb_data['review'].apply(remove_stopwords)

{'her', "don't", 'just', 'most', 'of', 'being', 'couldn', 'has', 'been', 'ain', 'himself', 'above', 'mustn', 'against', 'o', 'each', 'over', 'she', 'they', 'a', 'than', 'yours', "shouldn't", 'own', 'itself', 'why', 'myself', 'all', 'is', 'hasn', 'about', 'then', 'were', 'it', 'you', 'did', 'weren', 'an', "mightn't", 'ourselves', 'both', 'now', 'wouldn', 'and', 'some', 'we', 'there', "weren't", "you're", "shan't", 'after', 'll', "haven't", 's', 'for', "wouldn't", 'how', 'm', 'whom', 'haven', 'further', "hasn't", "you'll", 'at', 'up', 'are', 'mightn', 'too', 'can', 'wasn', 'its', 'few', 'through', 'ours', 'will', 'once', 'when', 'nor', "should've", 'your', 'because', 'not', 'here', 'during', 'while', 'between', 'that', 'out', "isn't", 'he', 'from', "hadn't", "she's", 'doing', 'as', 'themselves', 'theirs', 'same', 'didn', "that'll", 'won', 'to', 'hadn', 'shan', 'having', 'his', 'very', 'by', 've', 'on', "you'd", 'does', 'any', 'their', 'until', 'my', "needn't", 'd', 'needn', 'ma', "you've

In [15]:
#normalized train reviews
norm_train_reviews=imdb_data.review[:40000]
norm_train_reviews[0]

'one review ha mention watch 1 oz episod youll hook right thi exactli happen meth first thing struck oz wa brutal unflinch scene violenc set right word go trust thi show faint heart timid thi show pull punch regard drug sex violenc hardcor classic use wordit call oz nicknam given oswald maximum secur state penitentari focus mainli emerald citi experiment section prison cell glass front face inward privaci high agenda em citi home manyaryan muslim gangsta latino christian italian irish moreso scuffl death stare dodgi deal shadi agreement never far awayi would say main appeal show due fact goe show wouldnt dare forget pretti pictur paint mainstream audienc forget charm forget romanceoz doesnt mess around first episod ever saw struck nasti wa surreal couldnt say wa readi watch develop tast oz got accustom high level graphic violenc violenc injustic crook guard wholl sold nickel inmat wholl kill order get away well manner middl class inmat turn prison bitch due lack street skill prison exp

In [16]:
#Normalized test reviews
norm_test_reviews=imdb_data.review[40000:]
norm_test_reviews[40000]

'first want say lean liber polit scale found movi offens manag watch whole doggon disgrac film thi movi bring low origin idea ye wa origin thu 2 star instead 1 film writer uncr onli come thi act wa horribl charact unlik part lead ladi stori good qualiti made bf sort bad guy see mayb miss someth knowh wa earth relev charact movi shell ani money thi garbag almost wish peta would come rescu thi aw offens movi form protest disgust say anymor'

In [17]:
#Count vectorizer for bag of words
cv=CountVectorizer()
#transformed train reviews
cv_train_reviews=cv.fit_transform(norm_train_reviews)
#transformed test reviews
cv_test_reviews=cv.transform(norm_test_reviews)

print('BOW_cv_train:',cv_train_reviews.shape)
print('BOW_cv_test:',cv_test_reviews.shape)
#vocab=cv.get_feature_names()-toget feature names

BOW_cv_train: (40000, 156136)
BOW_cv_test: (10000, 156136)


In [18]:
#Tfidf vectorizer
tv=TfidfVectorizer()
#transformed train reviews
tv_train_reviews=tv.fit_transform(norm_train_reviews)
#transformed test reviews
tv_test_reviews=tv.transform(norm_test_reviews)
print('Tfidf_train:',tv_train_reviews.shape)
print('Tfidf_test:',tv_test_reviews.shape)

Tfidf_train: (40000, 156136)
Tfidf_test: (10000, 156136)


In [19]:
#labeling the sentient data
lb=LabelBinarizer()
#transformed sentiment data
sentiment_data=lb.fit_transform(imdb_data['sentiment'])
print(sentiment_data.shape)

(50000, 1)


In [20]:
#training the model
mnb=MultinomialNB()
#fitting the svm for bag of words
mnb_bow=mnb.fit(cv_train_reviews,train_sentiments)
print(mnb_bow)
#fitting the svm for tfidf features
mnb_tfidf=mnb.fit(tv_train_reviews,train_sentiments)
print(mnb_tfidf)

MultinomialNB()
MultinomialNB()


In [21]:
#Predicting the model for bag of words
mnb_bow_predict=mnb.predict(cv_test_reviews)
print(mnb_bow_predict)
#Predicting the model for tfidf features
mnb_tfidf_predict=mnb.predict(tv_test_reviews)
print(mnb_tfidf_predict)

['negative' 'negative' 'negative' ... 'negative' 'negative' 'negative']
['negative' 'negative' 'negative' ... 'negative' 'negative' 'negative']


In [22]:
#Accuracy of the model
#Accuracy score for bag of words
mnb_bow_score=accuracy_score(test_sentiments,mnb_bow_predict)
print("mnb_bow_score :",mnb_bow_score)
#Accuracy score for tfidf features
mnb_tfidf_score=accuracy_score(test_sentiments,mnb_tfidf_predict)
print("mnb_tfidf_score :",mnb_tfidf_score)

mnb_bow_score : 0.8532
mnb_tfidf_score : 0.8626


In [23]:
#Classification report for bag of words
mnb_bow_report=classification_report(test_sentiments,mnb_bow_predict,target_names=['Positive','Negative'])
print(mnb_bow_report)
#Classification report for tfidf features
mnb_tfidf_report=classification_report(test_sentiments,mnb_tfidf_predict,target_names=['Positive','Negative'])
print(mnb_tfidf_report)

              precision    recall  f1-score   support

    Positive       0.82      0.91      0.86      4993
    Negative       0.90      0.79      0.84      5007

    accuracy                           0.85     10000
   macro avg       0.86      0.85      0.85     10000
weighted avg       0.86      0.85      0.85     10000

              precision    recall  f1-score   support

    Positive       0.85      0.88      0.86      4993
    Negative       0.88      0.85      0.86      5007

    accuracy                           0.86     10000
   macro avg       0.86      0.86      0.86     10000
weighted avg       0.86      0.86      0.86     10000



In [26]:
import warnings
warnings.filterwarnings('ignore')
from sklearn.datasets import fetch_20newsgroups

newsgroups_train = fetch_20newsgroups(subset='train', shuffle=True, remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', shuffle=True, remove=('headers', 'footers', 'quotes'))


HTTPError: HTTP Error 403: Forbidden

In [ ]:
newsgroups_train.target_names

In [ ]:
print(newsgroups_train.DESCR)


In [ ]:
newsgroups_train.data[:3]


In [ ]:

import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer,SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(300)

In [ ]:
import nltk
nltk.download('wordnet')


In [ ]:
print(WordNetLemmatizer().lemmatize('sent',pos='v'))

In [ ]:

import pandas as pd
stemmer=SnowballStemmer('english')
original_words=['caresses','flies','dies','mules','denied','died','agreed','owned',
 'humbled','sized','meeting','stating','siezing','itemization','sensational',
 'traditional','reference','colonizer','plotted']
singles=[stemmer.stem(plural) for plural in original_words]
pd.DataFrame(data={'Original word':original_words,'stemmed':singles})

In [ ]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text,pos='v'))
def preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text):
    if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) >3:
    result.append(lemmatize_stemming(token))

    return result

In [ ]:
document=50
doc_sample='This disk had faild many times.I would like to get it replaced'
print('Original Document: ')
words=[]
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n Tokenized and lemmatized document: ')
print(preprocess(doc_sample))

In [ ]:
processed_docs=[]
for doc in newsgroups_train.data:
    processed_docs.append(preprocess(doc))
    print(processed_docs[:2])

In [ ]:

dictionary=gensim.corpora.Dictionary(processed_docs)

In [ ]:
count =0
for k,v in dictionary.iteritems():
    print(k,v)
    count+=1
    if count >15:
    break

In [ ]:
dictionary.filter_extremes(no_below=15,no_above=0.1,keep_n=100000)

In [ ]:
bow_corpus=[dictionary.doc2bow(doc) for doc in processed_docs]
document_num=25
bow_doc_x=bow_corpus[document_num]
for i in range(len(bow_doc_x)):
    print('Word {} (\"{}\") appears {} time:'.format(bow_doc_x[i][0],
    dictionary[bow_doc_x[i][0]],
    bow_doc_x[i][1]))


In [ ]:
lda_model=gensim.models.LdaMulticore(bow_corpus,num_topics=8,id2word=dictionary,
 passes=10)

In [ ]:
for idx,topic in lda_model.print_topics(-1):
    print('Topic: {} \n Words: {}'.format(idx,topic))
    print('\n')